In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


In [1]:
df = spark.read.format("csv").option("header","true").load("Files/products/products.csv")
# df now is a Spark DataFrame containing CSV data from "Files/products/products.csv".
display(df)

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6eec6008-acd3-4c84-9aa9-5a5539c6ca51)

In [2]:
df.write.format("delta").saveAsTable("managed_products")

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 4, Finished, Available)

In [6]:
df.write.format("delta").saveAsTable("external_products", path="abfss://dp_fabric@onelake.dfs.fabric.microsoft.com/fabric_lakehouse.Lakehouse/Files/external_products")

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 8, Finished, Available)

In [7]:
%%sql

DESCRIBE FORMATTED managed_products;

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 9, Finished, Available)

<Spark SQL result set with 12 rows and 3 fields>

In [8]:
%%sql

DESCRIBE FORMATTED external_products;

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 10, Finished, Available)

<Spark SQL result set with 12 rows and 3 fields>

In [9]:
%%sql

DROP TABLE managed_products;
DROP TABLE external_products;

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [10]:
%%sql

CREATE TABLE products
USING DELTA
LOCATION 'Files/external_products';

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 13, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [11]:
%%sql

SELECT * FROM products;

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 14, Finished, Available)

<Spark SQL result set with 295 rows and 4 fields>

In [12]:
%%sql

UPDATE products
SET ListPrice = ListPrice * 0.9
WHERE Category = 'Mountain Bikes';

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 15, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [13]:
%%sql

DESCRIBE HISTORY products;

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 16, Finished, Available)

<Spark SQL result set with 2 rows and 15 fields>

In [14]:
delta_table_path = 'Files/external_products'

# Get the current data
current_data = spark.read.format("delta").load(delta_table_path)
display(current_data)

# Get the version 0 data
original_data = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
display(original_data)

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, 89f3995d-d5e1-4a05-91c5-61511ad0ad9c)

SynapseWidget(Synapse.DataFrame, 6f9de5f4-f241-43a3-bd75-b757b1c06a2a)

In [15]:
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a folder
inputPath = 'Files/data/'
mssparkutils.fs.mkdirs(inputPath)

# Create a stream that reads data from the folder, using a JSON schema
jsonSchema = StructType([
StructField("device", StringType(), False),
StructField("status", StringType(), False)
])
iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

# Write some event data to the folder
device_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''
mssparkutils.fs.put(inputPath + "data.txt", device_data, True)
print("Source stream created...")

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 18, Finished, Available)

Source stream created...


In [16]:
# Write the stream to a delta table
delta_stream_table_path = 'Tables/iotdevicedata'
checkpointpath = 'Files/delta/checkpoint'
deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
print("Streaming to delta sink...")

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 19, Finished, Available)

Streaming to delta sink...


In [17]:
%%sql

SELECT * FROM IotDeviceData;

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 20, Finished, Available)

<Spark SQL result set with 9 rows and 2 fields>

In [18]:
# Add more data to the source stream
more_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 21, Finished, Available)

True

In [19]:
%%sql

SELECT * FROM IotDeviceData;

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 22, Finished, Available)

<Spark SQL result set with 16 rows and 2 fields>

In [20]:
deltastream.stop()

StatementMeta(, 25960fdf-5234-4583-bdbc-8d9d0cf929bb, 23, Finished, Available)